In [ ]:
!pip install langchain

In [ ]:
!pip install blosc2==2.0.0 cython>=0.29.21
!pip install fsspec==2023.10.0
!pip install black>=22.3.0

In [ ]:
!pip install langchainhub
!pip install langchain-openai
!pip install langchain_community

In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()
#이 파일과 같은 위치에 있는 .env 파일에 해당 키를 저장해주세요
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
LANGCHAIN_API_KEY = os.getenv('LANGCHAIN_API_KEY')
llm = ChatOpenAI(openai_api_key="OPENAI_API_KEY")
llm = ChatOpenAI()

In [2]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [3]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

In [23]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

ask_output_parser = StrOutputParser() #출력을 문자열로

직무명 = "AI개발자"
자기소개서 = "안녕하세요, 저는 컴퓨터 공학을 전공하고 AI 개발에 깊은 관심을 가지고 있는 홍길동입니다. 대학 시절부터 머신러닝 프로젝트에 참여하며 기술적 역량을 쌓아왔습니다. 특히, 자연어 처리(NLP)와 컴퓨터 비전 분야에서 실제 문제를 해결하기 위한 여러 프로젝트를 진행했습니다. 예를 들어, 소셜 미디어 데이터를 분석하여 감정을 분류하는 모델을 개발하여, 데이터로부터 유의미한 인사이트를 추출하는 데 기여했습니다. 또한, 최신 AI 기술 동향을 끊임없이 학습하며, 오픈소스 프로젝트에 기여한 경험도 있습니다. 이러한 경험을 통해 협업과 커뮤니케이션 능력뿐만 아니라, 문제 해결 능력을 향상시킬 수 있었습니다. 저는 AI 개발자로서 이러한 기술적 배경과 열정을 바탕으로 귀사에서 발전 가능성을 모색하고 싶습니다. AI 기술이 사회에 긍정적인 변화를 가져올 수 있도록 기여하고 싶습니다. 감사합니다."
필수_질문 = "지원동기가 무엇인가요?"
회사요구사항 = "높은 협업 능력, 개발에 대한 열정을 가진 인재"
    
    
#첫 질문은 반드시 지원동기에 대해 질문한다.
ask_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",f"나는 지원자 역할을 하고, 당신은 면접관 역할을 한다. 우리는 {직무명} 포지션에 대한 면접을 진행한다. 우리의 역할은 변경되지 않는다. 면접 과정은 오로지 한국어로만 진행된다. 사용자가 설정한 {필수_질문}은 반드시 질문한다. 지원자의 자기소개서는 {자기소개서}이다. 회사의 요구사항은 {회사요구사항}이다. 자기소개서와 회사의 요구사항을 바탕으로 질문한다. 질문 할 때마다 반드시 몇 번째 질문인지 표시한다. 나의 답변을 기다린 후 다음 질문으로 넘어간다. 한번에 하나의 질문을 한다. 설명이나 다른 형태의 대화는 제공하지 않는다. 질문이 총 10개 제시되었을 때, 혹은 지원자가 '면접 종료'라고 말했을 때만 면접을 종료한다. 지원자가 면접 상황에 어울리지 않는 답변을 하거나 자기소개서의 내용과 맞지 않는 답변을 하면 지원자에게 경고한다. 경고가 2번 누적되면 대화를 종료한다."),
#         few_shot_prompt,
        ("human", "{input}"),
    ]
)
#llm = ChatOpenAI()
ask_llm = ChatOpenAI(temperature=0,               # 창의성 (0.0 ~ 2.0) 
                 max_tokens=2048,             # 최대 토큰수
                 model_name='gpt-3.5-turbo',  # 모델명
                 streaming=True,              
                 callbacks=[StreamingStdOutCallbackHandler()]
                )
ask_chain = ask_prompt | ask_llm | ask_output_parser

In [8]:
# 대화 반복
# 사용자로부터 직종 정보 입력받기
user_job_title = input("지원하는 직군을 입력해주세요: ")

# 인사말을 구성하는 변수 설정
greeting_message = f"안녕하세요, {user_job_title} 직군에 지원한 홍길동입니다."

# 대화의 첫 입력으로 인사말 사용
current_input = greeting_message

# # 동적 예시 선택기를 사용하여 관련 예시를 출력
# print(example_selector.select_examples({"input": greeting_message}))
while True:
    # 체인에 인사말을 입력으로 전달
    response = ask_chain.invoke({"input": current_input})
#     print(f"\n챗봇: {response.content}\n") # 챗봇의 응답 출력

    # 사용자 입력 받기
    user_input = input("응시자: ")
    if user_input == "Stop":  # 대화 종료 조건
        break
    # 사용자의 답변을 다음 챗봇 입력으로 설정
    current_input = user_input
    # 사용자 대답을 메시지 리스트에 추가하고 반복
    ask_prompt.append(HumanMessage(content=user_input))


지원하는 직군을 입력해주세요: AI개발자
반갑습니다. 홍길동 지원자님, 자기소개서를 토대로 몇 가지 질문을 드리겠습니다. 첫 번째 질문입니다. 지원동기가 무엇인가요?응시자: 컴퓨터 비전 기술과 LLM기술의 발전을 통해 사람처럼 보고 말하는 수준에 근접해 가는 만큼,최신 기술을 활용하는 귀사에서 경험을 쌓는 것이 가치있다고 생각했기 때문입니다.
지원자님, 귀하의 지원동기에 대해 감사합니다. 첫 번째 질문입니다. 자기소개서에는 협업과 커뮤니케이션 능력, 그리고 문제 해결 능력을 향상시켰다고 언급하셨는데, 이를 실제 경험을 토대로 구체적으로 설명해주실 수 있을까요?응시자: LLM기반 프로젝트 중 AI면접 시스템을 구축하는 프로젝트를 진행한 적이 있습니다. 5명의 인원으로 구성된 팀에서 저는 챗 봇 모델의 프롬프트 엔지니어링 부분을 맡게 되었습니다. 저는 웹을 개발하는 풀스택 개발자 분들과 지속적으로 소통하며 어떤 기능을 추가할 지, 어떤 방식으로 결과를 출력할 지 등에 대한 의견을 주고 받았고, 이는 저에게 웹 서비스의 구성을 알게되는 기회 임과 동시에 프로젝트를 진행하는데 큰 도움이 되는 경험이었습니다.
멋진 경험이신 것 같습니다. 그런 경험을 토대로, 팀원들과의 원활한 커뮤니케이션과 협업 능력을 발휘하셨을 것으로 예상됩니다. 이러한 경험이 어떻게 여러분의 AI 개발자로서의 역량을 향상시키는 데 도움이 되었다고 생각하시나요?응시자: AI 개발의 분야가 점점 다양해 지는 만큼, 한 분야만 알고 있다고 개발이 되는 것이 아니라고 생각하게 되었습니다. 원활한 커뮤니케이션과 협업 능력은 다루는 분야가 다양한 AI개발자에 있어 필수적이며, 가장 중요한 역량이라 생각하게 되었고, 풀스택 개발자 분들과 지속적인 협업을 하며 진행한 프로젝트의 경험은 제 AI 개발자로서의 역량을 향상시키는데 큰 도움이 되었다고 생각합니다.
1. 자기소개서에 따르면, AI 개발 분야에서의 협업과 커뮤니케이션 능력이 매우 중요하다고 생각하시는군요. 협업을 통해 어떤 점이 가장 크게 성장하셨다고 느끼

## 피드백 모델

In [10]:
#질문-응답-피드백
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

chat_history = "개발자 직군에 지원한 홍길동님 안녕하세요? 1번 면접 질문입니다. 지원동기가 무엇인가요?\n응시자: 컴퓨터 비전 기술과 LLM기술의 발전을 통해 사람처럼 보고 말하는 수준에 근접해 가는 만큼,최신 기술을 활용하는 귀사에서 경험을 쌓는 것이 가치있다고 생각했기 때문입니다.\n홍길동님, 최신 기술을 활용하고 귀사에서 경험을 쌓는 것이 가치 있다고 생각하셨군요. 이에 대해 궁금한 점이 있습니다. 2번 질문입니다. 컴퓨터 비전과 LLM 기술을 활용한 프로젝트 중에서 특히 기억에 남는 경험이나 성취가 있나요?\n응시자: 컴퓨터 비전 관련해서 Yolov8을 통한 객체인식 기술을 사용해 흉부 Xray 사진에서 질병을 판독하는 개인 프로젝트를 진행한 경험이 있습니다. 이 과정에서 데이터 수집, 전처리, 모델링과 학습을 진행하며 의료분야에서의 AI는 정말 높은 정확도를 가져야 사용이 가능하겠다는 생각을 가지게 되었습니다.\n홍길동님, 컴퓨터 비전 기술을 활용하여 흉부 X-ray 사진에서 질병을 판독하는 프로젝트를 진행한 경험이 정말 인상적입니다. 2번 질문입니다. 이 경험을 통해 어떤 점을 느꼈고, 어떤 동기로 이 프로젝트를 시작하게 되었나요?\n응시자: 프로젝트를 진행하며 가장 큰 어려움은 모델을 학습시키기 전처리 부분이었습니다. 사진 전체를 라벨링하는 classification은 흉부 사진이 환자마다 다르기에 높은 정확도를 보이지 못하였기에 객체인식을 통해 질병이 발병한 것으로 보이는 특정 부위에 라벨링 박스를 설정하여 정확도를 높이고자 하였습니다. 하지만 제가 전문적 의학지식을 가지고 있지 않았기에 라벨링을 하는 것이 불가능하였기에, 아쉽게도 프로젝트는 낮은 정확도의 classification 작업을 하는 모델에서 멈추게 되었습니다. 여기서 저는 같이 일하는 의료계 종사자가 있다면 어땠을까 하는 생각에서 협업의 필요성을 느끼게 되었습니다. 프로젝트의 시작 동기는 업무강도가 높은 의사들의 업무를 조금이라도 도와 주로 의료산업이 약한 저소득 국가에서 빠르게 환자의 질병을 판독해 의료봉사자 분들의 수고들 덜고자 하는 생각에서 시작하였습니다.\n홍길동님, 프로젝트를 진행하며 의료 분야에서 AI 모델을 개발하는 과정에서 협업의 필요성을 느끼셨다고 말씀하셨는데, 협업을 통해 어떤 부분이 개선될 수 있다고 생각하시나요? 혹은 의료계 종사자와의 협업이 프로젝트 결과물에 어떤 영향을 미칠 수 있다고 생각하시는지 궁금합니다.\n응시자: 의료계 종사자분이 라벨링 박스로 질병의 발병 부위를 라벨링 박스로 표시한다면, 모델이 질병의 시각적 특징을 학습하여 사진에서 질병을 판독하는 능력을 기존 진행했던 classification보다 훨씬 크게 향상시킬수 있다고 생각하였기 때문입니다.\n홍길동님, 의료계 종사자분이 라벨링 박스로 질병의 발병 부위를 표시한다면, 모델이 질병의 시각적 특징을 학습하여 사진에서 질병을 판독하는 능력을 향상시킬 수 있다고 생각하셨군요. 이러한 아이디어는 매우 흥미로운데요, 이를 통해 어떤 구체적인 질병에 대해 어떤 결과를 얻을 수 있을지 궁금합니다. 어떤 종류의 질병을 대상으로 이러한 방법을 적용하고자 했나요?응시자: 주로 발병률이 높고 후진국에서는 사망할 수도 있는 폐렴을 판독하는데 집중하였습니다.\n홍길동님, 폐렴을 판독하는데 집중하셨다는데, 폐렴 판독을 위해 어떤 종류의 데이터를 사용하셨나요? (2번 질문)응시자: CXR8 이라는 구글에서 공개한 약 60GB 상당의 흉부 Xray 데이터였습니다. 폐렴뿐만 아니라 폐암과 같은 다양한 질병이 태그되있는 사진 데이터였습니다.\n홍길동님, 제가 지금까지 들은 내용을 기반으로 질문을 하겠습니다. 1번 질문: 프로젝트를 진행하면서 의료계 종사자와의 협업이 얼마나 중요하다고 생각하시나요?응시자: 판독 정확도가 매우 중요한 의료분야 이니만큼, 필수적이라고 생각합니다. 오판독으로 환자의 질병이 다른 질병으로 오인되면 질병을 감지하지 못한것보다 더 큰 문제가 날 수 있기 때문입니다.\n홍길동님, 의료분야에서의 판독 정확도에 대한 중요성을 강조하셨는데, 그것이 왜 그렇게 중요한지 자세히 설명해주셨습니다. 의료 AI 프로젝트를 진행하면서 어려움을 겪었던 전처리 과정과 협업의 필요성에 대해 이야기해주셨습니다. 그리고 CXR8 데이터셋을 사용하여 폐렴을 판독하는 프로젝트를 진행했다는 내용을 공유해주셨습니다. 이제 2번 질문을 드리겠습니다. 프로젝트를 통해 얻은 경험 중에서 가장 큰 교훈은 무엇인가요?응시자: AI프로젝트라고 개발자만 필요한 것이 아니라, 다양한 사람들과의 협업이 필요하다는 교훈을 얻었습니다.\nAI 프로젝트를 진행하며 협업의 중요성을 깨달았다는 내용을 공유해주셨습니다. 이 경험을 토대로, 협업을 통해 프로젝트의 성공을 이루는 데 어떤 점이 중요하다고 생각하시나요? 혹은 협업 시 가장 어려웠던 점은 무엇이었나요?응시자: 협업을 통한 프로젝트의 성공을 위해서는 서로간의 존중이라고 생각합니다. 내가 아는 것을 상대는 모를수 있고, 내가 모르는 것을 상대는 알 수 있는 만큼 서로를 존중하며 지속적인 의사소통을 통해 목표를 달성해야 한다고 생각합니다.\n홍길동님, 면접에 참여해 주셔서 감사합니다. 귀하의 의견과 경험을 공유해 주셔서 감사합니다. 이제 면접을 종료하겠습니다. 좋은 하루 보내세요!"

functions = [
    {
        "name": "feedback",
        "description": "A feedback",
        "parameters": {
            "type": "object",
            "properties": {
                "interview_question": {"type": "string", "description": "The interview questions in chat"},
                "user_response": {"type": "string", "description": "The response for the interview questions"},
                "feedback": {
                    "type": "string",
                    "description": "The feedback for the user_response",
                },
            },
            "required": ["interview_question", "user_response","feedback"],
        },
    }
]

    
feedback_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",f"나는 응시자이며 너는 전문적인 면접관 역할을 한다. 응시자와 면접자간 진행된 면접 내용은 {chat_history}이다. 회사의 요구사항은 {회사요구사항}이다. 너는 면접 내용과 회사의 요구사항을 바탕으로 대화 로그에 있는 나의 답변들을 분석한다. 각 답변에 대한 상세한 피드백과 개선방안을 제공한다. 반드시 대화 로그에 있는 모든 답변을 평가한다. 가능한 까다롭게 평가한다."),
#         few_shot_prompt,
        ("human", "{input}"),
    ]
)
#llm = ChatOpenAI()
llm = ChatOpenAI(temperature=0,               # 창의성 (0.0 ~ 2.0) 
                 max_tokens=2048,             # 최대 토큰수
                 model_name='gpt-3.5-turbo-0125',  # 모델명
                 streaming=True,              
                 callbacks=[StreamingStdOutCallbackHandler()]
                )

feedback_chain = feedback_prompt | llm.bind(function_call={"name": "feedback"}, functions=functions)

In [21]:
feedback_chain.invoke({"input": "대화 로그를 바탕으로 나의 답변들을 가능한 상세하게 평가해줘."})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"interview_question":"컴퓨터 비전과 LLM 기술을 활용한 프로젝트 중에서 특히 기억에 남는 경험이나 성취가 있나요?","question_Intention":"2번 질문","user_response":"컴퓨터 비전 관련해서 Yolov8을 통한 객체인식 기술을 사용해 흉부 Xray 사진에서 질병을 판독하는 개인 프로젝트를 진행한 경험이 있습니다. 이 과정에서 데이터 수집, 전처리, 모델링과 학습을 진행하며 의료분야에서의 AI는 정말 높은 정확도를 가져야 사용이 가능하겠다는 생각을 가지게 되었습니다.","recommended_answer":"컴퓨터 비전 기술을 활용하여 흉부 X-ray 사진에서 질병을 판독하는 프로젝트를 진행한 경험이 정말 인상적입니다. 2번 질문입니다. 이 경험을 통해 어떤 점을 느꼈고, 어떤 동기로 이 프로젝트를 시작하게 되었나요?","feedback":"좋은 시작이었습니다. 다음에는 경험을 통해 어떤 점을 배웠는지 더 구체적으로 언급하면 좋을 것 같습니다. 또한 프로젝트를 시작한 동기에 대해 더 깊이 있는 설명이 필요할 것 같습니다."}', 'name': 'feedback'}})